# P1 Part A: Markov decision processes and Algorithms


# Q3

 There are four states (A, B, C, D) and two actions (L for leave, S for stay/go back). To calculate the state value function Vπ(s) under the policy π = {S, S, S, S}, we can use the Bellman equation for the given policy. 

 First, let's define the reward matrix R and transition probability matrices P_L an P_S as given in the figures. Then, we will implement the policy evaluation step of policy iteration to find Vπ​(s) for each state. We'll assume a certain discount factor β, which is typically less than 1.

In [1]:
import numpy as np

In [2]:
# Define the transition probability matrices for actions L and S
P_L = np.array([
    [0.0, 0.2, 0.3, 0.5],
    [0.5, 0.0, 0.5, 0.0],
    [0.0, 0.5, 0.0, 0.5],
    [0.0, 0.5, 0.5, 0.0]
])

P_S = np.array([
    [1.0, 0.0, 0.0, 0.0],
    [0.2, 0.8, 0.0, 0.0],
    [0.0, 0.2, 0.8, 0.0],
    [0.0, 0.2, 0.8, 0.0]
])

# Define the reward matrix for actions L and S
R = np.array([
    [2, 3],
    [1, 2],
    [3, 1],
    [10, 0]
])


In [3]:
# Define the policy π = {S, S, S, S} for each state
policy = np.array([1, 1, 1, 1])  # 0 for L, 1 for S

# Assume a discount factor beta
beta = 0.9

# Initialize the state value function V
V = np.zeros(4)

def compute_state_value_function(policy, V, P_L, P_S, R, beta):
    """
    Compute the state value function Vπ(s) for a given policy π.
    """
    threshold = 1e-10
    while True:
        delta = 0
        for s in range(4):
            v = V[s]
            if policy[s] == 0:
                # If policy is to leave (L)
                V[s] = sum(P_L[s, s_prime] * (R[s_prime, 0] + beta * V[s_prime]) for s_prime in range(4))
            else:
                # If policy is to stay/go back (S)
                V[s] = sum(P_S[s, s_prime] * (R[s_prime, 1] + beta * V[s_prime]) for s_prime in range(4))
            delta = max(delta, abs(v - V[s]))
        if delta < threshold:
            break
    return V

# Compute the state value function Vπ(s) for the given policy
V_pi = compute_state_value_function(policy, V, P_L, P_S, R, beta)
print("State value function Vπ(s):", V_pi)

def policy_iteration(P_L, P_S, R, beta):
    """
    Perform policy iteration to find the optimal policy.
    """
    policy = np.zeros(4, dtype=int)
    V = np.zeros(4)
    is_policy_stable = False
    while not is_policy_stable:
        V = compute_state_value_function(policy, V, P_L, P_S, R, beta)
        is_policy_stable = True
        for s in range(4):
            old_action = policy[s]
            # Look-ahead one step for each action and choose the best one
            action_values = [
                sum(P_L[s, s_prime] * (R[s_prime, 0] + beta * V[s_prime]) for s_prime in range(4)),
                sum(P_S[s, s_prime] * (R[s_prime, 1] + beta * V[s_prime]) for s_prime in range(4))
            ]
            policy[s] = np.argmax(action_values)
            if old_action != policy[s]:
                is_policy_stable = False
    return policy, V

# Perform policy iteration
optimal_policy, optimal_V = policy_iteration(P_L, P_S, R, beta)
print("Optimal policy:", ["L" if p == 0 else "S" for p in optimal_policy])
print("Optimal state value function:", optimal_V)

def value_iteration(P_L, P_S, R, beta):
    """
    Perform value iteration to find the optimal value function.
    """
    V = np.zeros(4)
    threshold = 1e-10
    while True:
        delta = 0
        for s in range(4):
            v = V[s]
            # Look-ahead one step for each action and choose the best one
            V[s] = max(
                sum(P_L[s, s_prime] * (R[s_prime, 0] + beta * V[s_prime]) for s_prime in range(4)),
                sum(P_S[s, s_prime] * (R[s_prime, 1] + beta * V[s_prime]) for s_prime in range(4))
            )
            delta = max(delta, abs(v - V[s]))
        if delta < threshold:
            break
    return V

# Perform value iteration
optimal_V_value_iteration = value_iteration(P_L, P_S, R, beta)
print("Optimal state value function from value iteration:", optimal_V_value_iteration)


State value function Vπ(s): [30.         27.14285714 21.73469388 21.73469388]
Optimal policy: ['L', 'L', 'L', 'L']
Optimal state value function: [40.31333999 38.38491667 39.43091928 37.01712618]
Optimal state value function from value iteration: [40.31333999 38.38491667 39.43091928 37.01712618]
